In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tensorflow import keras
### hack tf-keras to appear as top level keras
import sys
sys.modules['keras'] = keras
### end of hack

import keras.backend as K
from keras.utils import multi_gpu_model
from keras.callbacks import TensorBoard

from sklearn.model_selection import KFold

import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, callbacks
from keras.callbacks import *
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
import numpy as np
import pprint
import os
from os import listdir
from os.path import isfile, join
from datetime import datetime

# Importing custom framework
import sys, os
sys.path.insert(0, os.path.abspath('../..'))
from data_split import prepare_data_spectrogram

In [2]:
def build_cnn():
    model = models.Sequential()

    model.add(layers.Conv2D(227, (3, 1), activation='relu', input_shape=(1, 227, 227), data_format='channels_first'))
    model.add(layers.MaxPooling2D((3, 1), 2))
    
    model.add(layers.Conv2D(64, (3, 1), activation='relu'))
    model.add(layers.MaxPooling2D((2, 1), 2))
    
    model.add(layers.Conv2D(128, (3, 1), activation='relu'))
    model.add(layers.MaxPooling2D((2, 1), 2))
    
    model.add(layers.Conv2D(128, (3, 1), activation='relu'))
    model.add(layers.MaxPooling2D((2, 1), 2))
    
    model.add(layers.Conv2D(128, (3, 1), activation='relu'))
    model.add(layers.MaxPooling2D((2, 1), 2))
    
    #Fully Connected Part
    model.add(layers.Flatten())
    
    model.add(layers.Dense(1024))#, input_dim=1024
    model.add(layers.Dropout(0.5, seed=1337))
    model.add(layers.Activation('relu'))
    
    model.add(layers.Dense(512))#, input_dim=1024
    model.add(layers.Dropout(0.5, seed=1337))
    model.add(layers.Activation('relu'))
    
    model.add(layers.Dense(10, input_dim=512))
    model.add(layers.Activation('softmax'))
    return model

In [3]:
spectrogramPath = '../../merged/spectrogram.hdf5'
X_train, X_val, X_test, y_train, y_val, y_test = prepare_data_spectrogram(spectrogramPath, False)

In [4]:
# Decrease training data size to make training feasable
X_temp = []
y_temp = []
n = 300
count_dict = dict.fromkeys(range(0, 10), 0)
for i in range(len(y_train)):
    curr = y_train[i]
    if count_dict[curr] >= n:
        finished = True
        for x in count_dict.values():
            if x < 300:
                finished = False
                break
        if finished == True:
            break
    else:
        count_dict[curr] = count_dict[curr] + 1
        X_temp.append(X_train[i])
        y_temp.append(curr)
        
X_train = np.array(X_temp)
y_train = np.array(y_temp)
print(X_train.shape)

(3000, 1, 227, 227)


In [ ]:
tf.reset_default_graph()

tbCallBack = TensorBoard(log_dir='logs', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=100,
                         write_images=True)

earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

opt = SGD(lr=0.001, momentum=0.9, clipvalue=5)

num_folds = 5

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# Merge inputs and targets
Xs = np.concatenate((X_train, X_val), axis=0)
ys = np.concatenate((y_train, y_val), axis=0)
Xs = X_train
ys = y_train

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=False)
acc_per_fold = []
loss_per_fold = []
fold_no = 0

In [ ]:
print(Xs.shape)

(3000, 1, 227, 227)


In [ ]:
#K.clear_session()
model = build_cnn()
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

filepath = "logs/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
for train,val in kfold.split(Xs, ys):
    print(Xs[train].shape)
    print(ys[train].shape)
    training_history = model.fit(Xs[train], ys[train], epochs=100, validation_split=0.2, callbacks=[earlyStopping, tbCallBack], shuffle=False)
    # Generate generalization metrics
    scores = model.evaluate(Xs[val], ys[val], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    
val_scores = model.evaluate(X_val, y_val, verbose=0)
print(val_score)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(2400, 1, 227, 227)
(2400,)
Train on 1920 samples, validate on 480 samples
Epoch 1/100
1920/1920 [==============================] - 3889s 2s/sample - loss: 2.4047 - acc: 0.0979 - val_loss: 2.3034 - val_acc: 0.1083
Epoch 2/100
1920/1920 [==============================] - 3241s 2s/sample - loss: 2.3078 - acc: 0.0974 - val_loss: 2.3031 - val_acc: 0.0938
Epoch 3/100
1920/1920 [==============================] - 3584s 2s/sample - loss: 2.3030 - acc: 0.1057 - val_loss: 2.3021 - val_acc: 0.1000
Epoch 4/100
1920/1920 [==============================] - 3377s 2s/sample - loss: 2.3029 - acc: 0.0974 - val_loss: 2.3023 - val_acc: 0.1042
Epoch 5/100
1920/1920 [==============================] - 3388s 2s/sample - loss: 2.3021 - acc: 0.0969 - val_loss: 2.3018 - val_acc: 0.1083
Epoch 6/100
1920/1920 [==============================] - 3870s 2s/sample - loss: 2.3018 - acc: 0.1078 - val_loss: 2

In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')